In [1]:
from Bio.PDB import PDBParser, Superimposer, PDBIO

In [43]:
parser = PDBParser(QUIET=True)
reference_structure = parser.get_structure('reference', 'example_complexes/top_2.pdb')
new_rbd_structure = parser.get_structure('new_rbd', 'chains/rbd_WT_6w41_C.pdb')
new_vhh_structure = parser.get_structure('new_vhh_7', 'chains/vhh_7_by_sabpred_api_new.pdb')

In [44]:
reference_rbd = reference_structure[0]['F']
reference_vhh = reference_structure[0]['H']
new_rbd = new_rbd_structure[0]['C']
new_vhh = new_vhh_structure[0]['H']

In [45]:
# Извлечение Cα атомов для выравнивания
def get_ca_atoms(chain):
    ca_atoms = []
    for residue in chain:
        if 'CA' in residue:
            ca_atoms.append(residue['CA'])
    return ca_atoms

reference_rbd_ca = get_ca_atoms(reference_rbd)
new_rbd_ca = get_ca_atoms(new_rbd)
reference_vhh_ca = get_ca_atoms(reference_vhh)
new_vhh_ca = get_ca_atoms(new_vhh)

In [46]:
# Выравнивание нового RBD по старому, используя только общие остатки
common_residues_rbd = min(len(reference_rbd_ca), len(new_rbd_ca))
superimposer = Superimposer()
superimposer.set_atoms(reference_rbd_ca[:common_residues_rbd], new_rbd_ca[:common_residues_rbd])
superimposer.apply(new_rbd.get_atoms())

In [47]:
# Выравнивание нового VHH по старому, используя только общие остатки
common_residues = min(len(reference_vhh_ca), len(new_vhh_ca))
superimposer.set_atoms(reference_vhh_ca[:common_residues], new_vhh_ca[:common_residues])
superimposer.apply(new_vhh.get_atoms())

In [48]:
# Изменение идентификаторов цепей
new_rbd.id = 'C'
new_vhh.id = 'D'

# Создание нового комплекса
new_complex = reference_structure.copy()
new_complex[0].add(new_rbd)
new_complex[0].add(new_vhh)

In [49]:
# Удаление старых цепей
del new_complex[0]['H']
del new_complex[0]['F']

In [50]:
new_complex[0]['C'].id = 'C'
new_complex[0]['D'].id = 'H'

In [51]:
# Сохранение нового комплекса
io = PDBIO()
io.set_structure(new_complex)
io.save('example_complexes/new_complex_3.pdb')

In [54]:
def replace_vhh(reference_pdb, new_vhh_pdb, output_pdb, reference_vhh_chain='H', new_vhh_chain='H'):
    # Загрузка структур
    parser = PDBParser(QUIET=True)
    reference_structure = parser.get_structure('reference', reference_pdb)
    new_vhh_structure = parser.get_structure('new_vhh', new_vhh_pdb)

    # Извлечение цепей
    reference_vhh = reference_structure[0][reference_vhh_chain]
    new_vhh = new_vhh_structure[0][new_vhh_chain]

    # Извлечение Cα атомов для выравнивания
    def get_ca_atoms(chain):
        ca_atoms = []
        for residue in chain:
            if 'CA' in residue:
                ca_atoms.append(residue['CA'])
        return ca_atoms

    reference_vhh_ca = get_ca_atoms(reference_vhh)
    new_vhh_ca = get_ca_atoms(new_vhh)

    # Выравнивание нового VHH по старому, используя только общие остатки
    common_residues_vhh = min(len(reference_vhh_ca), len(new_vhh_ca))
    superimposer = Superimposer()
    superimposer.set_atoms(reference_vhh_ca[:common_residues_vhh], new_vhh_ca[:common_residues_vhh])
    superimposer.apply(new_vhh.get_atoms())

    # Изменение идентификатора цепи
    new_vhh.id = 'C'

    # Создание нового комплекса
    new_complex = reference_structure.copy()
    new_complex[0].add(new_vhh)

    # Удаление старой цепи
    del new_complex[0][reference_vhh_chain]

    # Обратная переименовка цепи
    new_complex[0]['C'].id = reference_vhh_chain

    # Сохранение нового комплекса
    io = PDBIO()
    io.set_structure(new_complex)
    io.save(output_pdb)

In [55]:
replace_vhh('example_complexes/top_2.pdb', 'chains/vhh_7_by_sabpred_api_new.pdb', 'example_complexes/new_complex_4.pdb')

In [2]:
def replace_rbd(reference_pdb, new_rbd_pdb, output_pdb, reference_rbd_chain='A', new_rbd_chain='A'):
    # Загрузка структур
    parser = PDBParser(QUIET=True)
    reference_structure = parser.get_structure('reference', reference_pdb)
    new_rbd_structure = parser.get_structure('new_rbd', new_rbd_pdb)

    # Извлечение цепей
    reference_rbd = reference_structure[0][reference_rbd_chain]
    new_rbd = new_rbd_structure[0][new_rbd_chain]

    # Извлечение Cα атомов для выравнивания
    def get_ca_atoms(chain):
        ca_atoms = []
        for residue in chain:
            if 'CA' in residue:
                ca_atoms.append(residue['CA'])
        return ca_atoms

    reference_rbd_ca = get_ca_atoms(reference_rbd)
    new_rbd_ca = get_ca_atoms(new_rbd)

    # Выравнивание нового RBD по старому, используя только общие остатки
    common_residues_rbd = min(len(reference_rbd_ca), len(new_rbd_ca))
    superimposer = Superimposer()
    superimposer.set_atoms(reference_rbd_ca[:common_residues_rbd], new_rbd_ca[:common_residues_rbd])
    superimposer.apply(new_rbd.get_atoms())

    # Изменение идентификатора цепи
    new_rbd.id = 'X'

    # Создание нового комплекса
    new_complex = reference_structure.copy()
    new_complex[0].add(new_rbd)

    # Удаление старой цепи
    del new_complex[0][reference_rbd_chain]

    # Обратная переименовка цепи
    new_complex[0]['X'].id = reference_rbd_chain

    # Сохранение нового комплекса
    io = PDBIO()
    io.set_structure(new_complex)
    io.save(output_pdb)

In [58]:
replace_rbd('example_complexes/top_2.pdb', 'chains/rbd_BQ.1.1_8fxc_E.pdb', 'example_complexes/new_complex_5.pdb', "F", "E")

In [5]:
import os

ref_complex = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/reference_pdbs/refs/rbd_WU_6vw1_E_vhh_7.pdb"
ref_complex_rbd_ch = "E"
ref_complex_vhh_ch = "H"

vhh_name = "vhh_7"


directory_path = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/RBDs"

for file in os.listdir(directory_path):
    if ("rbd_" in file) and (".pdb" in file):
        splited_filename = file.split("_")
        new_rbd_ch = splited_filename[-1].replace(".pdb", "")

        name_prefix = file.replace(".pdb", "")
        output_filename = f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/reference_pdbs/rbd_vhh7/{name_prefix}_{vhh_name}.pdb"

        replace_rbd(reference_pdb=ref_complex, new_rbd_pdb=f"{directory_path}/{file}", output_pdb=output_filename, reference_rbd_chain=ref_complex_rbd_ch, new_rbd_chain=new_rbd_ch)